# SHACL

- SHACL = Shapes Constraint Language
- język do opisu i walidacji grafów RDF
- https://www.w3.org/TR/shacl/

## Przykład

zob. https://www.w3.org/TR/shacl/#shacl-example


## pySHACL


A Python validator for SHACL.

https://github.com/RDFLib/pySHACL

## Python Module Use
For basic use of this module, you can just call the `validate` function of the `pyshacl` module like this:

```python
from pyshacl import validate
r = validate(data_graph,
      shacl_graph=sg,
      ont_graph=og,
      inference='rdfs',
      abort_on_error=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)
conforms, results_graph, results_text = r
```

Where:
* `data_graph` is an rdflib `Graph` object or file path of the graph to be validated
* `shacl_graph` is an rdflib `Graph` object or file path or Web URL of the graph containing the SHACL shapes to validate with, or None if the SHACL shapes are included in the data_graph.
* `ont_graph` is an rdflib `Graph` object or file path or Web URL a graph containing extra ontological information, or None if not required.
* `inference` is a Python string value to indicate whether or not to perform OWL inferencing expansion of the `data_graph` before validation.
Options are 'rdfs', 'owlrl', 'both', or 'none'. The default is 'none'.
* `abort_on_error` (optional) a Python `bool` value to indicate whether or not the program should abort after encountering a validation error or to continue. Default is to continue.
* `meta_shacl` (optional) a Python `bool` value to indicate whether or not the program should enable the Meta-SHACL feature. Default is False.
* `advanced`: (optional) a Python `bool` value to enable SHACL Advanced Features
* `js`: (optional) a Python `bool` value to enable SHACL-JS Features (if `pyshacl[js]` is installed)
* `debug` (optional) a Python `bool` value to indicate whether or not the program should emit debugging output text, including violations that didn't lead to non-conformance overall. So when debug is True don't judge conformance by absense of violation messages. Default is False.

Some other optional keyword variables available available on the `validate` function:
* `data_graph_format`: Override the format detection for the given data graph source file.
* `shacl_graph_format`: Override the format detection for the given shacl graph source file.
* `ont_graph_format`: Override the format detection for the given extra ontology graph source file.
* `do_owl_imports`: Enable the feature to allow the import of subgraphs using `owl:imports` for the shapes graph and the ontology graph. Note, you explicitly cannot use this on the target data graph.
* `serialize_report_graph`: Convert the report results_graph into a serialised representation (for example, 'turtle')
* `check_dash_result`: Check the validation result against the given expected DASH test suite result.
* `check_sht_result`: Check the validation result against the given expected SHT test suite result.

Return value:
* a three-component `tuple` containing:
  * `conforms`: a `bool`, indicating whether or not the `data_graph` conforms to the `shacl_graph`
  * `results_graph`: a `Graph` object built according to the SHACL specification's [Validation Report](https://www.w3.org/TR/shacl/#validation-report) structure
  * `results_text`: python string representing a verbose textual representation of the [Validation Report](https://www.w3.org/TR/shacl/#validation-report)


In [83]:
# !pip install pyshacl

In [3]:
from pyld import jsonld
import json
from rdflib.namespace import SKOS, DCTERMS, DC
from rdflib import Graph, plugin
from rdflib.serializer import Serializer
import pyshacl
from pyshacl import validate

In [6]:
g = Graph()
g.parse("https://spec.edmcouncil.org/fibo/ontology/master/latest/FBC/FunctionalEntities/NorthAmericanEntities/USExampleIndividuals/")
print(len(g))
og = Graph()
og.load("https://spec.edmcouncil.org/fibo/ontology/master/latest/BE/LegalEntities/CorporateBodies.rdf")
print(len(og))
# g += og
# print(len(g))

1901
148


In [22]:
shacl_shapes = '''
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix rts: <http://ontology.roche.com/> .
@prefix sh:	<http://www.w3.org/ns/shacl#>.
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix fibo-be-le-cb: <https://spec.edmcouncil.org/fibo/ontology/BE/LegalEntities/CorporateBodies/> .
@prefix fibo-fnd-rel-rel: <https://spec.edmcouncil.org/fibo/ontology/FND/Relations/Relations/> .
@prefix fibo-be-le-lp: <https://spec.edmcouncil.org/fibo/ontology/BE/LegalEntities/LegalPersons/> .
@prefix fibo-be-le-lei: <https://spec.edmcouncil.org/fibo/ontology/BE/LegalEntities/LEIEntities/> .
@prefix fibo-fnd-plc-adr: <https://spec.edmcouncil.org/fibo/ontology/FND/Places/Addresses/> .

fibo-be-le-lp:LegalEntityShape a sh:NodeShape ;
    sh:nodeKind sh:IRI ;
    sh:targetClass fibo-be-le-cb:ForProfitCorporation ;
    sh:property [
        sh:path fibo-fnd-rel-rel:hasLegalName ;
        sh:minCount 2 ;
        sh:datatype xsd:string
        ] ;
    sh:property [
        sh:path fibo-be-le-lei:hasLegalAddress ;
        sh:minCount 1 ;
        sh:maxCount 1 ;
        sh:class fibo-fnd-plc-adr:ConventionalStreetAddress ;
        sh:nodeKind sh:IRI ;
        ] ;        
    sh:closed false .
'''

In [23]:
sg = Graph().parse(data=shacl_shapes, format="ttl")

In [9]:
data_graph = g
r = validate(data_graph, 
             shacl_graph=sg, 
             ont_graph=og, 
             inference='both', 
             abort_on_error=False, 
             meta_shacl=False, 
             advanced=False, 
             js=False, 
             debug=False)

conforms, results_graph, results_text = r

# print(results_text)

In [10]:
print(conforms)

False


In [11]:
print(results_graph.serialize(format="ttl").decode("utf-8"))

@prefix fibo-fnd-rel-rel: <https://spec.edmcouncil.org/fibo/ontology/FND/Relations/Relations/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <https://spec.edmcouncil.org/fibo/ontology/FBC/FunctionalEntities/NorthAmericanEntities/USExampleIndividuals/StateStreetCorporation-US-MA> ;
            sh:resultMessage "Less than 2 values on fibo-fbc-fct-usind:StateStreetCorporation-US-MA->fibo-fnd-rel-rel:hasLegalName" ;
            sh:resultPath fibo-fnd-rel-rel:hasLegalName ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MinCountConstraintComponent ;
            sh:sourceShape _:ub1bL13C17 ],
        [ a sh:ValidationResult ;
            sh:focusNode <https://spec.edmcouncil.org/fibo/ontology/FBC/FunctionalEntities/NorthAmericanEntities/USExampleIndividuals/WellsFargoAndCompany-US-DE> 

In [12]:
print(results_text)

Validation Report
Conforms: False
Results (5):
Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:datatype xsd:string ; sh:minCount Literal("2", datatype=xsd:integer) ; sh:path fibo-fnd-rel-rel:hasLegalName ]
	Focus Node: fibo-fbc-fct-usind:JPMorganChaseAndCo-US-DE
	Result Path: fibo-fnd-rel-rel:hasLegalName
	Message: Less than 2 values on fibo-fbc-fct-usind:JPMorganChaseAndCo-US-DE->fibo-fnd-rel-rel:hasLegalName
Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:datatype xsd:string ; sh:minCount Literal("2", datatype=xsd:integer) ; sh:path fibo-fnd-rel-rel:hasLegalName ]
	Focus Node: fibo-fbc-fct-usind:BankOfNewYorkMellonCorporation-US-DE
	Result Path: fibo-fnd-rel-rel:hasLegalName
	Message: Less than 2 values on fibo-fbc-fct-usind:BankOfNewYorkMellonCorporation-US-DE->fibo-fnd-rel-

In [13]:
g.bind("skos",SKOS)
g.bind("dcterms", DCTERMS)
g.bind("dc", DC)
results_graph.serialize("results_graph.ttl", format="ttl")

## Ćwiczenie

In [ ]:
fibo_onts = ["https://spec.edmcouncil.org/fibo/ontology/master/latest/BE/LegalEntities/CorporateBodies/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/BE/Corporations/Corporations/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/BE/LegalEntities/FormalBusinessOrganizations/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/BE/LegalEntities/LEIEntities/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/BE/LegalEntities/LegalPersons/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/BE/OwnershipAndControl/Executives/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/BE/OwnershipAndControl/OwnershipParties/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/BE/OwnershipAndControl/CorporateControl/,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/FBC/FunctionalEntities/FinancialServicesEntities/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/FBC/FunctionalEntities/BusinessRegistries/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/FND/Accounting/AccountingEquity/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/FND/AgentsAndPeople/People/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/FND/Arrangements/Documents/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/FND/Arrangements/Ratings/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/FND/Parties/Parties/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/FND/Law/Jurisdiction/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/FND/Parties/Roles/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/FND/Organizations/FormalOrganizations/,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/FND/Places/Addresses/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/FND/Places/Locations/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/FND/DatesAndTimes/FinancialDates/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/FND/ProductsAndServices/ProductsAndServices/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/FND/Relations/Relations/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/FND/TransactionsExt/MarketTransactions/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/LOAN/LoanContracts/LoanCore/" ,
"https://spec.edmcouncil.org/fibo/ontology/master/latest/SEC/Securities/SecuritiesListings/" ,
"https://www.omg.org/spec/LCC/Countries/ISO3166-1-CountryCodes/" ,
"https://www.omg.org/spec/LCC/Countries/Regions/ISO3166-2-SubdivisionCodes-US/" ,
"https://www.omg.org/spec/LCC/Languages/LanguageRepresentation/"]